In [2]:
import numpy as np
def fun(x: np.ndarray):
    return x**2

fun(np.array([1,2,3]))

array([1, 4, 9])

In [8]:
# 有shock發生時
import numpy as np
from numpy.linalg import inv
class Demand:
    def __init__(self, D, h):
        self.D=D()
        self.h=h
    def quantity(self, pd):
        qd=np.array((self.D@pd)+self.h)
        return qd
class Supply:
    def __init__(self, C, e):
        self.C=C
        self.e=e
    def quantity(self, ps):
        qs=np.array((self.C@ps)+self.e)
        return qs 
class Market:
    def __init__(self, demand, supply):
        self.demand=demand
        self.supply=supply
    def price(self):
        pe=inv(self.supply.C-self.demand.D)@(self.demand.h-self.supply.e)
        return pe
    def quantity(self):
        qe=self.demand.D@self.price()+self.demand.h
        return qe
    def equlibrium(self):
        pe=self.price()
        qe=self.quantity()
        return pe, qe

D=np.array([
    [-10,-5],
    [-1,-10]
])
h=np.array([
    [100],
    [50]
])
C=np.array([
    [10,5],
    [5,10]
])
e=np.array([
    [0],
    [0]
])

demand = Demand(D, h)
supply = Supply(C, e)
market = Market(demand, supply)

market.equlibrium()

(array([[4.41176471],
        [1.17647059]]),
 array([[50.        ],
        [33.82352941]]))

In [1]:
# 有shock發生時
import numpy as np
from numpy.linalg import inv
class Demand:
    def __init__(self, D, h):
        self.D=D
        self.h=h
    def quantity(self, pd):
        qd=np.array((self.D@pd)+self.h)
        return qd
class Supply:
    def __init__(self, C, e):
        self.C=C
        self.e=e
    def quantity(self, ps):
        qs=np.array((self.C@ps)+self.e)
        return qs 
class Market:
    def __init__(self, demand, supply):
        self.demand=demand
        self.supply=supply
    def price(self):
        D=self.demand.D
        h=self.demand.h
        C=self.supply.C
        e=self.supply.e
        pe=inv(C-D)@(h-e)
        return pe
    def quantity(self):
        D=self.demand.D
        h=self.demand.h
        qe=D@self.price()+h
        return qe
    def equilibrium(self):
        pe=self.price()
        qe=self.quantity()
        return pe, qe

D=np.array([
    [-10,-5],
    [-1,-10]
])
h=np.array([
    [100],
    [50]
])
C=np.array([
    [10,5],
    [5,10]
])
e=np.array([
    [0],
    [0]
])

demand = Demand(D, h)
supply = Supply(C, e)
market = Market(demand, supply)

market.equilibrium()

(array([[4.41176471],
        [1.17647059]]),
 array([[50.        ],
        [33.82352941]]))

In [2]:
# 改變參數
demand.h = np.array([[80],[50]])
market.equilibrium()

(array([[3.23529412],
        [1.52941176]]),
 array([[40.        ],
        [31.47058824]]))

In [14]:
import numpy as np
from numpy.linalg import inv
class Demand:
    def __init__(self, D, h):
        self.D=D
        self.h=h
    def quantity(self, pd):
        qd=np.array((self.D@pd)+self.h)
        return qd
class Supply:
    def __init__(self, C, e0):
        self.C=C
        self.e0=e0
    def quantity(self, ps):
        qs=np.array((self.C@ps)+self.e0)
        return qs
class DynamicMarket:
    def __init__(self, demand, supply, e0:np.ndarray):
        self.demand = demand
        self.supply = supply
        self.e0 = e0
        self.newe = e0
        self.p=inv(self.supply.C-self.demand.D)@(self.demand.h-e0)
        self.q=self.demand.D@self.p+self.demand.h
        self.equilibrium_dynamics = [[e0, self.p, self.q]]
    def price(self):
        D=self.demand.D
        h=self.demand.h
        C=self.supply.C
        e0=self.supply.e0
        pe=inv(C-D)@(h-e0)
        return pe
    def quantity(self):
        D=self.demand.D
        h=self.demand.h
        qe=D@self.price()+h
        return qe
    def equilibrium(self):
        pe=self.price()
        qe=self.quantity()
        return pe, qe
    def generate_onePeriod_equilibrium(self):
        """
        return a tuple of (e, p, q)
        and track the dynamics in self.equilibrium_dynamics
        """
        epsilon = self.supply_shock(0, 0.4)
        data=np.array(epsilon(2))
        newe=data.reshape(2,1)

        newp=inv(self.supply.C-self.demand.D)@(self.demand.h-self.newe)
        newq=self.demand.D@newp+self.demand.h
        
        # keep track of the dynamics
        self.equilibrium_dynamics[len(self.equilibrium_dynamics)-1][0] = newe
        self.equilibrium_dynamics[len(self.equilibrium_dynamics)-1][1] = newp
        self.equilibrium_dynamics[len(self.equilibrium_dynamics)-1][2] = newq
        self.equilibrium_dynamics.append([newe, newp, newq])

        # set the market time to the next period 
        self.newe = newe 
        return (newe, newp, newq)
    def supply_shock(self, mu, sigma):
        def draw(size):
            return np.random.normal(mu, sigma, size)
        return draw

In [15]:
D=np.array([
    [-10,-5],
    [-1,-10]
])
h=np.array([
    [100],
    [50]
])
C=np.array([
    [10,5],
    [5,10]
])
e0=np.array([
    [0],
    [0]
])


demand = Demand(D, h)
supply = Supply(C, e0)
market = DynamicMarket(demand, supply, e0)

In [20]:
# initial state 
market.e0, market.equilibrium_dynamics

(array([[0],
        [0]]),
 [[array([[-0.10276874],
          [ 0.35819541]]),
   array([[4.41176471],
          [1.17647059]]),
   array([[50.        ],
          [33.82352941]])],
  [array([[-0.39383363],
          [-0.11586857]]),
   array([[4.42834508],
          [1.1535867 ]]),
   array([[49.94861563],
          [34.03578788]])],
  [array([[ 0.2028407],
          [-0.4753472]]),
   array([[4.43152349],
          [1.17633638]]),
   array([[49.80308318],
          [33.8051127 ]])],
  [array([[ 0.92060574],
          [-0.13669469]]),
   array([[4.3858521 ],
          [1.20801173]]),
   array([[50.10142035],
          [33.5340306 ]])],
  [array([[ 0.18556011],
          [-0.37722704]]),
   array([[4.353591  ],
          [1.20075744]]),
   array([[50.46030287],
          [33.63883465]])],
  [array([[-0.08558338],
          [ 0.00327969]]),
   array([[4.38975449],
          [1.201935  ]]),
   array([[50.09278006],
          [33.59089546]])],
  [array([[-0.00171745],
          [ 0.48255

In [21]:
market.generate_onePeriod_equilibrium()

(array([[-0.10448903],
        [ 0.24772028]]),
 array([[4.43054502],
        [1.13038036]]),
 array([[50.042648  ],
        [34.26565141]]))

In [22]:
# simulate the dynamics for 10 periods
for t in range(3):
    _ = market.generate_onePeriod_equilibrium()

market.equilibrium_dynamics

[[array([[-0.10276874],
         [ 0.35819541]]),
  array([[4.41176471],
         [1.17647059]]),
  array([[50.        ],
         [33.82352941]])],
 [array([[-0.39383363],
         [-0.11586857]]),
  array([[4.42834508],
         [1.1535867 ]]),
  array([[49.94861563],
         [34.03578788]])],
 [array([[ 0.2028407],
         [-0.4753472]]),
  array([[4.43152349],
         [1.17633638]]),
  array([[49.80308318],
         [33.8051127 ]])],
 [array([[ 0.92060574],
         [-0.13669469]]),
  array([[4.3858521 ],
         [1.20801173]]),
  array([[50.10142035],
         [33.5340306 ]])],
 [array([[ 0.18556011],
         [-0.37722704]]),
  array([[4.353591  ],
         [1.20075744]]),
  array([[50.46030287],
         [33.63883465]])],
 [array([[-0.08558338],
         [ 0.00327969]]),
  array([[4.38975449],
         [1.201935  ]]),
  array([[50.09278006],
         [33.59089546]])],
 [array([[-0.00171745],
         [ 0.48255253]]),
  array([[4.41689548],
         [1.17476737]]),
  array([[

In [31]:
# Exercise AR(1) process
import numpy as np
class AR1:
    def __init__(self, rho, x0):
        self.rho = rho
        self.x0 = x0
        self.x = x0
        self.simulated_dynamics = [[x0]]
    def simulate_onePeriod(self):
        """
        return a tuple of (xnext)
        and track the dynamics in self.simulate_dynamics
        """ 
        sigma = 0.2
        epsilon = np.random.normal(0, sigma, 1)
        
        xnext = self.rho*self.x+epsilon

        # keep track of the dynamics
        self.simulated_dynamics.append([xnext])

        # set the market time to the next period
        self.x = xnext
        return (xnext)

In [32]:
ar1=AR1(rho=0.9, x0=0.1)
ar1.simulate_onePeriod()

array([-0.20340062])

In [33]:
# simulate the dynamics for 10 periods
for t in range(10):
    _ = ar1.simulate_onePeriod()

ar1.simulated_dynamics

[[0.1],
 [array([-0.20340062])],
 [array([0.01624787])],
 [array([-0.29801737])],
 [array([-0.22693158])],
 [array([-0.13716362])],
 [array([-0.40536743])],
 [array([-0.26254683])],
 [array([0.03268552])],
 [array([0.11150864])],
 [array([0.1174883])],
 [array([0.13105394])]]